In [1]:
import nglview
from openff.toolkit import ForceField, Molecule, Topology
from openff.units import unit

from openff.interchange import Interchange
from openff.interchange.components._packmol import UNIT_CUBE, pack_box
from openff.interchange.drivers import get_summary_data

In [2]:
molecules = [
    Molecule.from_smiles(smi)
    for smi in ["ClC(Cl)(Cl)Cl", "CC1=CC=CC=C1", "CS(=O)C", "CCCCCCO"]
]

In [3]:
topology = pack_box(
    molecules=molecules,
    number_of_copies=4 * [200],
    mass_density=850 * unit.kilogram / unit.meter**3,
    box_shape=UNIT_CUBE,
)

In [4]:
topology.to_file("system.pdb")
nglview.show_structure_file("system.pdb")

NGLWidget()

In [5]:
force_field = ForceField("openff_unconstrained-2.1.0.offxml")
interchange = Interchange.from_smirnoff(force_field=force_field, topology=topology)

In [6]:
interchange.visualize("nglview")

NGLWidget()

In [7]:
get_summary_data(interchange, _engines=["OpenMM", "GROMACS", "Amber"])

,Bond,Angle,Torsion,Electrostatics,vdW
OpenMM,7294.737428,23614.974229,6413.856957,-18759.309105,137408.193219
Amber,7294.736219,23614.974650,6413.856942,273049.698114,141648.815410


In [11]:
import time

import openmm

# Length of the simulation.
num_steps = 1000  # number of integration steps to run

# Logging options.
trj_freq = 10  # number of steps per written trajectory frame
data_freq = 10  # number of steps per written simulation statistics

# Integration options
time_step = 2 * openmm.unit.femtoseconds  # simulation timestep
temperature = 300 * openmm.unit.kelvin  # simulation temperature
friction = 1 / openmm.unit.picosecond  # friction constant

integrator = openmm.LangevinIntegrator(temperature, friction, time_step)

In [12]:
simulation = interchange.to_openmm_simulation(integrator=integrator)

simulation.context.setVelocitiesToTemperature(temperature)

pdb_reporter = openmm.app.PDBReporter("trajectory.pdb", trj_freq)
state_data_reporter = openmm.app.StateDataReporter(
    "data.csv",
    data_freq,
    step=True,
    potentialEnergy=True,
    temperature=True,
    density=True,
)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(state_data_reporter)

In [13]:
print("Starting simulation")
start = time.process_time()

# Run the simulation
simulation.step(num_steps)

end = time.process_time()
print(f"Elapsed time {end - start} seconds")
print("Done!")

Starting simulation
Elapsed time 9.043640955 seconds
Done!


In [17]:
nglview.show_mdtraj(md.load("trajectory.pdb"))

NGLWidget(max_frame=99)